<a href="https://colab.research.google.com/github/flexycode/CCMACLRL_EXERCISES_COM231ML/blob/main/Lab/Lab-Exercise-6/Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 6: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all Regression models

Submit your results to:
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview



## Install required packages

In [27]:
pip install scikit-learn --quiet

## Import all necessary libraries

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

All libraries imported successfully!


# Load the datasets

## Dataset File

In [29]:
train_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/train.csv?raw=true'

test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/test.csv?raw=true'

## Load data

In [30]:
train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Training columns: {train_df.columns.tolist()}")

Training data shape: (1460, 81)
Test data shape: (1459, 80)
Training columns: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'O

## Test File

In [31]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [32]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

## Basic data exploration

In [33]:
print("Training Data Info:")
print(train_df.info())
print("\nMissing values in training data:")
print(train_df.isnull().sum().sort_values(ascending=False).head(20))

Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   i

## Sample Submission File

In [5]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [6]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int64  
 1   SalePrice  1459 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


# Effective Data Preprocessing

In [34]:
def simple_preprocess(train_df, test_df):
    # Separate target variable
    y = train_df['SalePrice']
    train_ids = train_df['Id']
    test_ids = test_df['Id']

    # Combine train and test for consistent preprocessing
    combined = pd.concat([train_df.drop(['Id', 'SalePrice'], axis=1),
                         test_df.drop('Id', axis=1)], axis=0)

    # Select only numerical columns for simplicity
    numerical_cols = combined.select_dtypes(include=[np.number]).columns.tolist()

    # Handle missing values in numerical columns
    for col in numerical_cols:
        combined[col].fillna(combined[col].median(), inplace=True)

    # Use only numerical features
    X_combined = combined[numerical_cols]

    # Split back to train and test
    X_train = X_combined.iloc[:len(train_df)]
    X_test = X_combined.iloc[len(train_df):]

    return X_train, X_test, y, train_ids, test_ids

# Preprocess the data

In [35]:
X_train, X_test, y_train, train_ids, test_ids = simple_preprocess(train_df, test_df)

print(f"Processed training features: {X_train.shape}")
print(f"Processed test features: {X_test.shape}")

Processed training features: (1460, 36)
Processed test features: (1459, 36)


## Split training data for validation


In [36]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Training split: {X_train_split.shape}")
print(f"Validation split: {X_val_split.shape}")

Training split: (1168, 36)
Validation split: (292, 36)


## 1. Train a KNN Regressor

In [37]:
print("=" * 50)
print("1. K-NEAREST NEIGHBORS REGRESSION")
print("=" * 50)

# Create and train KNN model
knn_model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=5))
])

knn_model.fit(X_train_split, y_train_split)

1. K-NEAREST NEIGHBORS REGRESSION


Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsRegressor())])

### Create and train KNN model

In [38]:
knn_model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor(n_neighbors=5))
])

knn_model.fit(X_train_split, y_train_split)

Pipeline(steps=[('scaler', StandardScaler()), ('knn', KNeighborsRegressor())])

### Make predictions

In [39]:
knn_pred = knn_model.predict(X_val_split)

### Calculate metrics

In [40]:
knn_rmse = np.sqrt(mean_squared_error(y_val_split, knn_pred))
knn_r2 = r2_score(y_val_split, knn_pred)

print(f"KNN Validation RMSE: {knn_rmse:,.2f}")
print(f"KNN Validation R² Score: {knn_r2:.4f}")

KNN Validation RMSE: 38,625.74
KNN Validation R² Score: 0.8055


- Perform cross validation

In [41]:
knn_cv_scores = cross_val_score(knn_model, X_train_split, y_train_split,
                               cv=5, scoring='neg_mean_squared_error')
knn_cv_rmse = np.sqrt(-knn_cv_scores.mean())

print("KNN Cross-Validation Results:")
print(f"Average RMSE: {knn_cv_rmse:,.2f}")
print(f"Standard Deviation: {np.sqrt(knn_cv_scores.std()):,.2f}")

KNN Cross-Validation Results:
Average RMSE: 38,829.19
Standard Deviation: 21,118.21


## 2. Train a SVM Regression

In [42]:
print("\n" + "=" * 50)
print("2. SUPPORT VECTOR MACHINE REGRESSION")
print("=" * 50)

# Create and train SVM model (using linear kernel for faster training)
svm_model = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='linear', C=1.0))
])

svm_model.fit(X_train_split, y_train_split)

# Make predictions
svm_pred = svm_model.predict(X_val_split)

# Calculate metrics
svm_rmse = np.sqrt(mean_squared_error(y_val_split, svm_pred))
svm_r2 = r2_score(y_val_split, svm_pred)

print(f"SVM Validation RMSE: {svm_rmse:,.2f}")
print(f"SVM Validation R² Score: {svm_r2:.4f}")


2. SUPPORT VECTOR MACHINE REGRESSION
SVM Validation RMSE: 83,466.12
SVM Validation R² Score: 0.0917


- Perform cross validation

In [10]:
svm_cv_scores = cross_val_score(svm_model, X_train_split, y_train_split,
                               cv=5, scoring='neg_mean_squared_error')
svm_cv_rmse = np.sqrt(-svm_cv_scores.mean())

print("SVM Cross-Validation Results:")
print(f"Average RMSE: {svm_cv_rmse:,.2f}")
print(f"Standard Deviation: {np.sqrt(svm_cv_scores.std()):,.2f}")

## 3. Train a Decision Tree Regression

In [43]:
print("\n" + "=" * 50)
print("3. DECISION TREE REGRESSION")
print("=" * 50)

# Create and train Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42, max_depth=10)
dt_model.fit(X_train_split, y_train_split)

# Make predictions
dt_pred = dt_model.predict(X_val_split)

# Calculate metrics
dt_rmse = np.sqrt(mean_squared_error(y_val_split, dt_pred))
dt_r2 = r2_score(y_val_split, dt_pred)

print(f"Decision Tree Validation RMSE: {dt_rmse:,.2f}")
print(f"Decision Tree Validation R² Score: {dt_r2:.4f}")


3. DECISION TREE REGRESSION
Decision Tree Validation RMSE: 38,164.17
Decision Tree Validation R² Score: 0.8101


- Perform cross validation

In [44]:
dt_cv_scores = cross_val_score(dt_model, X_train_split, y_train_split,
                              cv=5, scoring='neg_mean_squared_error')
dt_cv_rmse = np.sqrt(-dt_cv_scores.mean())

print("Decision Tree Cross-Validation Results:")
print(f"Average RMSE: {dt_cv_rmse:,.2f}")
print(f"Standard Deviation: {np.sqrt(dt_cv_scores.std()):,.2f}")

Decision Tree Cross-Validation Results:
Average RMSE: 48,842.15
Standard Deviation: 31,093.31


## 4. Train a Random Forest Regression

In [45]:
print("\n" + "=" * 50)
print("4. RANDOM FOREST REGRESSION")
print("=" * 50)

# Create and train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=15)
rf_model.fit(X_train_split, y_train_split)

# Make predictions
rf_pred = rf_model.predict(X_val_split)

# Calculate metrics
rf_rmse = np.sqrt(mean_squared_error(y_val_split, rf_pred))
rf_r2 = r2_score(y_val_split, rf_pred)

print(f"Random Forest Validation RMSE: {rf_rmse:,.2f}")
print(f"Random Forest Validation R² Score: {rf_r2:.4f}")


4. RANDOM FOREST REGRESSION
Random Forest Validation RMSE: 29,327.90
Random Forest Validation R² Score: 0.8879


### Perform cross validation for Random Forest

In [46]:
rf_cv_scores = cross_val_score(rf_model, X_train_split, y_train_split,
                              cv=5, scoring='neg_mean_squared_error')
rf_cv_rmse = np.sqrt(-rf_cv_scores.mean())

print("Random Forest Cross-Validation Results:")
print(f"Average RMSE: {rf_cv_rmse:,.2f}")
print(f"Standard Deviation: {np.sqrt(rf_cv_scores.std()):,.2f}")

Random Forest Cross-Validation Results:
Average RMSE: 31,640.86
Standard Deviation: 18,538.20


## 5. Compare all the performance of all regression models

In [49]:
print("\n" + "=" * 60)
print("5. MODEL PERFORMANCE COMPARISON")
print("=" * 60)

# Create comparison dataframe
models_comparison = pd.DataFrame({
    'Model': ['K-Nearest Neighbors', 'Support Vector Machine', 'Decision Tree', 'Random Forest'],
    'Validation_RMSE': [knn_rmse, svm_rmse, dt_rmse, rf_rmse],
    'Validation_R2': [knn_r2, svm_r2, dt_r2, rf_r2],
    'CV_RMSE': [knn_cv_rmse, svm_cv_rmse, dt_cv_rmse, rf_cv_rmse]
})

print(models_comparison.round(4))



5. MODEL PERFORMANCE COMPARISON


NameError: name 'svm_cv_rmse' is not defined

## 6. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [15]:
id = sf.pop('Id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'Id': id,
    'Sale_Price': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

NameError: name 'model' is not defined